<a href="https://colab.research.google.com/github/rcdavid1/dummy/blob/main/company25_in_class_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Dictionary

| Column Name | Description |
| :--- | :--- |
| **counterparty** | Name of the customer or business partner |
| **counterpartyidentifier** | Unique identifier code for the customer |
| **typeofsale** | Indicates whether the transaction is a "Sale" or "Return" |
| **documentnumber** | Invoice number |
| **invoicedate** | Date when the invoice was created |
| **referenceorder** | Sales order reference |
| **productnumber** | Product identifier |
| **productdescription** | Text description of the product |
| **productgroup** | Product group (highest level of grouping) |
| **productline** | Product line |
| **vendorproductnumber** | Vendor's reference number for the product |
| **manufacturer** | Name of the product manufacturer |
| **unitofmeasure** | Standardized unit of measure |
| **quantity** | Amount of product sold, rounded to 2 decimal places |
| **unit_price** | Price per unit, rounded to 2 decimal places |
| **unit_price_currency** | Currency code for the price (e.g., USD) |
| **total** | Calculated total value (quantity × unit price), rounded to 2 decimal places |
| **shiptoidentifier** | Ship-to identifier |
| **shiptoname** | Name of the ship-to location |
| **shiptostate** | State code of shipping destination |
| **shiptocounty** | Country code of shipping destination |
| **shipfromidentifier** | Plant/terminal identifier where product shipped from |
| **shipfromname** | Name of the shipping origin facility |
| **shipfromstate** | State code of shipping origin |
| **shipfromcounty** | Country code of shipping origin |

# Client's Description of Data and Request
* The dataset comprises sales information across all product groups for the calendar years 2021 to 2023.
* It includes details on product groups and product lines, facilitating product categorization. Although the data is anonymized, it effectively illustrates the seasonality of the agriculture cycle across all product groups.

## Download and Decrypt Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import sys
import os
sys.path.append('/content')

# Check if the directory exists before cloning
if not os.path.exists('company25_utils'):
    !git clone https://github.com/bsheese/company25_utils.git

from company25_utils.utils import download_and_decrypt_data

download_and_decrypt_data()

Cloning into 'company25_utils'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
Receiving objects: 100% (28/28), 13.20 KiB | 563.00 KiB/s, done.
remote: Total 28 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (13/13), done.


Downloading...
From (original): https://drive.google.com/uc?id=1omHwvtL4r1OGSUR12Th3RBpVliFxcnww
From (redirected): https://drive.google.com/uc?id=1omHwvtL4r1OGSUR12Th3RBpVliFxcnww&confirm=t&uuid=e27a36f3-e57d-4939-9ba8-2b7833788938
To: /content/encrypted_data.enc
100%|██████████| 287M/287M [00:05<00:00, 57.2MB/s]


Enter the password to decrypt the file: ··········
File decrypted successfully and saved as 'decrypted_data_file.csv'
The SHA-256 hash of encrypted_data.enc is:
f0fbaf55fadde70058539049d00b89fd5c00e5d1c68da30e3db57df937b87ac3


True

# Create Dataframe

## Initial Cleanup

In [ ]:
import pandas as pd
df = pd.read_csv(r'/content/decrypted_data_file.csv')
df.columns = df.columns.str.lower()

# shiptocounty and shipfromcounty, are monotonic (always US) and are dropped
df = df.drop(columns=['shiptocounty', 'shipfromcounty'])

# convert invoicedate to datetime, there is no hours, minutes, seconds, so only the date is extracted
df['invoicedate'] = pd.to_datetime(df['invoicedate'])

# resolving the single issues using CAD by coverting to USD
cad_mask = df.unit_price_currency == 'CAD'
print(f'# of rows with CAD: {cad_mask.sum()}')
cad_value = df.loc[cad_mask, 'unit_price'].values[0]
cad_date = df.loc[cad_mask, 'invoicedate'].values[0]

df.loc[cad_mask, 'unit_price'] = df.loc[cad_mask, 'unit_price'] * .78351  # the actual coversion is done here, the rest is just for display purposes
df = df.drop(columns = 'unit_price_currency')

# of rows with CAD: 1


# Your Code Starts Here

In [ ]:
# Use code cells for the code sections. Include comments occasionally to explain the intent of the code.

Use text cells for the written portions.